In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import random
import matplotlib.pyplot as plt

In [ ]:
# load dataset and split declare train and test data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

train = mnist.train.images 
test = mnist.test.images 

train_labels = mnist.train.labels
test_labels = mnist.test.labels

print("num train images: ", train.shape)
print("num test images: ", test.shape)

print("shape of train labels: ", train_labels.shape)

In [ ]:
# adjustable (hyper-)parameters
batch_size = 100
learning_rate = 0.0001 
learning_step = 30000
display_step= 300

# size of dataset
N = train.shape[0]
dim = train.shape[1]
dim_out = 10

In [ ]:
# method to get a batch of data
def next_batch(images, labels): 
    rand_indicies = random.sample(range(0, images.shape[0]), batch_size)
    
    return images[rand_indicies], labels[rand_indicies]

In [ ]:
# placeholder / variables
X = tf.placeholder(tf.float32, [None, dim]) 
y = tf.placeholder(tf.float32, [None, dim_out])

In [ ]:
# explicit network

W = tf.Variable(tf.zeros([dim, dim_out]))
b = tf.Variable(tf.zeros([dim_out]))

layer1 = tf.nn.softmax(tf.matmul(X, W) + b)

In [ ]:
# dense network
#     -> inputs, units, initializer, activation
#     -> https://www.tensorflow.org/api_docs/python/tf/layers/dense

def network(x): 
    layer1 = tf.layers.dense(inputs=x, units=dim_out, activation=tf.nn.relu)
    # do1 = tf.nn.dropout(x=layer1, keep_prob=0.2) 
    #layer2 = tf.layers.dense(inputs=layer1, units=dim_out, activation=tf.nn.softmax)
    
    return layer1

In [ ]:
pred = network(X)
#pred = tf.layers.dense(inputs=X, units=dim_out, activation=tf.nn.softmax)

#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=[1]))
cross_entropy = tf.losses.softmax_cross_entropy(y, pred)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    cost_hist = []
    acc_hist = []
    
    
    for step in range(learning_step):

        batch_xs, batch_ys = next_batch(train, train_labels)

        feed_dict = {X: batch_xs, y: batch_ys}
        
        acc, _ = sess.run((accuracy, optimizer), feed_dict=feed_dict)
        cost_hist.append(acc)
        
        if (step % display_step) == 0:
            
            feed_dict = {X: test, y: test_labels}
            
            acc = sess.run(accuracy, feed_dict=feed_dict)
            acc_hist.append(acc)
            
            print(step, acc)

    print("best accuracy: %f" % max(acc_hist))

In [ ]:
plt.plot(cost_hist)